In [1]:
import pandas as pd
import datetime
import numpy as np
import re
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.ensemble import RandomForestClassifier
#import xgboost
#from xgboost import XGBClassifier

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [134]:
# Function for calculating age of a person given his date of birth
from datetime import date
def calculate_age(born):
    #today = date.today()
    return 2019 - born.year
#- ((today.month, today.day) < (born.month, born.day))

In [135]:
#Function for calculating number of days between two dates 
def datediff(any_date):
    today = date.today()
    datediff = today-any_date 
    return datediff.days

In [136]:
# Calculating the Tenure in months given the data is in X Years and Y months format
def age_length(TenureAsYearsAndMonths):
    years = int(re.findall(r'(\d+)yrs',TenureAsYearsAndMonths )[0])
    months = int(re.findall(r'(\d+)mon',TenureAsYearsAndMonths )[0])
    total_tenure = (years*12)+months
    return total_tenure

In [137]:
def to_ordinal(PandasSeries,ColumnOrdinalDictionary):
    returning_value = []
    keys = [i for i in ColumnOrdinalDictionary]
    values = [i for i in ColumnOrdinalDictionary.values()]
    for i in range(len(PandasSeries)):
        if PandasSeries[i] in keys:
            returning_value.append(ColumnOrdinalDictionary[keys[keys.index(PandasSeries[i])]])
        else:
            returning_value.append(PandasSeries[i])
    return returning_value

In [138]:
# Dictionary for converting 'PERFORM_CNS.SCORE.DESCRIPTION' to ordinal data
Bureau_score_description ={ 'No Bureau History Available':'1',
                            'L-Very High Risk':'2',
                            'M-Very High Risk': '3',
                            'J-High Risk': '4',
                            'K-High Risk': '5',
                            'H-Medium Risk': '6',
                            'I-Medium Risk': '7',
                            'E-Low Risk': '8',
                            'F-Low Risk': '9',
                            'G-Low Risk': '10',
                            'A-Very Low Risk': '11',
                            'B-Very Low Risk': '12',
                            'C-Very Low Risk': '13',
                            'D-Very Low Risk': '14',
                            'Not Scored: No Activity seen on the customer (Inactive)': '15',
                            'Not Scored: Sufficient History Not Available': '16',
                            'Not Scored: Not Enough Info available on the customer': '17',
                            'Not Scored: No Updates available in last 36 months': '18',
                            'Not Scored: Only a Guarantor': '19',
                            'Not Scored: More than 50 active Accounts found': '20'}

In [139]:
# Function for doing all the data manipulations
def AllDataManipulations(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = to_ordinal(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],ColumnOrdinalDictionary=Bureau_score_description)
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES',
       'loan_default']],temp1,pd.DataFrame(temp2,columns = ['Bureau_score_description'])],axis=1)
    return Manip_Data,Manip_Completed

In [140]:
MasterData = pd.read_csv("train.csv")
Data1 = MasterData.copy()

In [141]:
Train_ManipData_1,Train_ManipCompleted_1 = AllDataManipulations(Data1)

In [142]:
Train_ManipCompleted_1.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Age,HowManyDaysSinceDisburse,State_ID,...,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AvgAcctAgeInMonths,CredHistLenInMonts,NO.OF_INQUIRIES,loan_default,EmploymentType_Salaried,EmploymentType_Self employed,Bureau_score_description
0,50578,58400,89.55,67,22807,45,1441,35,409,6,...,0,0,0,0,0,0,0,1,0,1
1,47145,65550,73.23,67,22807,45,1502,34,207,6,...,0,0,1,23,23,0,1,0,1,7
2,53278,61360,89.63,67,22807,45,1497,34,468,6,...,0,0,0,0,0,0,0,0,1,1
3,57513,66113,88.48,67,22807,45,1501,26,177,6,...,0,0,0,8,15,1,1,0,1,2
4,52378,60300,88.39,67,22807,45,1495,42,207,6,...,0,0,0,0,0,1,1,0,1,1


In [145]:
Train_ManipCompleted_1_Continuous = Train_ManipCompleted_1[['disbursed_amount', 'asset_cost', 'ltv','Age',
       'HowManyDaysSinceDisburse','PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES']]
Train_ManipCompleted_1_Categorical = Train_ManipCompleted_1[['branch_id', 'supplier_id','manufacturer_id', 'Current_pincode_ID',
                                       'State_ID', 'Employee_code_ID','MobileNo_Avl_Flag', 'Aadhar_flag', 
                                       'PAN_flag', 'VoterID_flag','Driving_flag', 'Passport_flag', 
                                       'loan_default','EmploymentType_Salaried', 
                                       'EmploymentType_Self employed','Bureau_score_description']]

In [146]:
Train_ManipCompleted_1['Age'].isnull().sum()

0

In [147]:
print(len(Train_ManipCompleted_1.columns))
print(len(Train_ManipCompleted_1_Continuous.columns)+len(Train_ManipCompleted_1_Categorical.columns))

41
41


In [148]:
len(Train_ManipCompleted_1_Continuous.columns)

25

In [149]:
# below are the columns if bucketed might help in good classification

#['disbursed_amount', 'asset_cost', 'ltv','Age',
#       'HowManyDaysSinceDisburse','PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
#       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
#       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
#       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
#       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
#       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
#       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
#       'CredHistLenInMonts', 'NO.OF_INQUIRIES']
#######################################

disbursed_amount_bucket_array = np.array([0,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,125000,150000,200000,250000,300000])
asset_cost_bucket_array = np.array([0,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,125000,150000,200000,250000,300000])
ltv_bucket_array = np.array([0,10,20,30,40,50,60,70,80,90,100])
Age_bucket_array = np.array([0,10,20,30,40,50,60,70,80,90,100,110])
HowManyDaysSinceDisburse = np.array([0,50,100,150,200,250,300,350,400,450,500,550,600,1000])
PERFORM_CNS_SCORE_bucket_array = np.array([0,100,200,300,400,500,600,700,800,900,1000,2000])
PRI_NO_OF_ACCTS_bucket_array = np.array([0,50,100,150,200,250,300,350,400,450,500,550,600,1000])
PRI_ACTIVE_ACCTS_bucket_array = np.array([0,50,100,150,200,250,300,350,400,450,500,550,600,1000])
PRI_OVERDUE_ACCTS_bucket_array = np.array([0,5,10,15,20,25,30])
# PRI.CURRENT.BALANCE is to be dealt with scaling only
# PRI.SANCTIONED.AMOUNT is to be dealt with scaling only
# PRI.DISBURSED.AMOUNT is to be deal with scaling only
SEC_NO_OF_ACCTS_bucket_array = np.array([0,10,20,30,40,50,60,70,80,90,100])
SEC_ACTIVE_ACCTS_bucket_array = np.array([0,10,20,30,40,50,60,70,80,90,100])
SEC_OVERDUE_ACCTS_bucket_array = np.array([0,5,10,15,20,25,30])
# SEC.CURRENT.BALANCE is to be dealt with scaling only.
# SEC.SANCTIONED.AMOUNT is to be dealt with scaling only
# SEC.DISBURSED.AMOUNT is to be dealt with scaling only
# PRIMARY.INSTAL.AMT  is to be dealt with scaling only
# SEC.INSTAL.AMT is to be dealt with scaling only
# NEW.ACCTS.IN.LAST.SIX.MONTHS No Scaling
# DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS - No Scaling
AvgAcctAgeInMonths_bucket_array = np.array([0,50,100,150,200,250,300,350,400,450,500,550,600,1000])
CredHistLenInMonts_bucket_array = np.array([0,50,100,150,200,250,300,350,400,450,500,550,600,1000])
# NO.OF_INQUIRIES - No Scaling

In [150]:
len(Train_ManipCompleted_1[Train_ManipCompleted_1['Age']<0])

19866

Train_ManipCompleted_1_Continuous['disbursed_amount'] = pd.cut(Train_ManipCompleted_1_Continuous['disbursed_amount'],
                                                               disbursed_amount_bucket_array,include_lowest=True,
                                                              labels = ['0To10000','10000To20000','20000To30000','30000To40000',
                                                                        '40000To50000','50000To60000','60000To70000','70000To80000',
                                                                        '80000To90000','90000To100000','100000To125000','125000To150000',
                                                                        '150000To200000','200000To250000','250000To300000'])
Train_ManipCompleted_1_Continuous['asset_cost'] = pd.cut(Train_ManipCompleted_1_Continuous['asset_cost'],
                                                         asset_cost_bucket_array,include_lowest=True,
                                                        labels = ['0To10000','10000To20000','20000To30000','30000To40000',
                                                                        '40000To50000','50000To60000','60000To70000','70000To80000',
                                                                        '80000To90000','90000To100000','100000To125000','125000To150000',
                                                                        '150000To200000','200000To250000','250000To300000'])
Train_ManipCompleted_1_Continuous['ltv'] = pd.cut(Train_ManipCompleted_1_Continuous['ltv'],ltv_bucket_array,include_lowest=True,
                                                 labels = ['0To10','10To20','20To30','30To40','40To50','50To60','60To70','70To80',
                                                           '80To90','90To100'])
Train_ManipCompleted_1_Continuous['Age'] = pd.cut(Train_ManipCompleted_1_Continuous['Age'],Age_bucket_array,include_lowest=True,
                                                 labels = ['0To10','10To20','20To30','30To40','40To50','50To60','60To70','70To80',
                                                           '80To90','90To100','100To110'])
Train_ManipCompleted_1_Continuous['HowManyDaysSinceDisburse'] = pd.cut(Train_ManipCompleted_1_Continuous['HowManyDaysSinceDisburse'],
                                                                       HowManyDaysSinceDisburse,include_lowest=True,
                                                                      labels = ['0To50','50To100','100To150','150To200','200To250',
                                                                                '250To300','300To350','350To400','400To450','450To500',
                                                                                '500To550','550To600','600To1000'])
Train_ManipCompleted_1_Continuous['PERFORM_CNS.SCORE'] = pd.cut(Train_ManipCompleted_1_Continuous['PERFORM_CNS.SCORE'],
                                                                PERFORM_CNS_SCORE_bucket_array,include_lowest=True,
                                                               labels = ['0To100','100To200','200To300','300To400','400To500',
                                                                         '500To600','600To700','700To800','800To900','900To1000','1000To2000'])
Train_ManipCompleted_1_Continuous['PRI.NO.OF.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.NO.OF.ACCTS'],
                                                              PRI_NO_OF_ACCTS_bucket_array,include_lowest=True,
                                                             labels = ['0To50','50To100','100To150','150To200','200To250',
                                                                                '250To300','300To350','350To400','400To450','450To500',
                                                                                '500To550','550To600','600To1000'])
Train_ManipCompleted_1_Continuous['PRI.ACTIVE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.ACTIVE.ACCTS'],
                                                               PRI_ACTIVE_ACCTS_bucket_array,include_lowest=True,
                                                              labels = ['0To50','50To100','100To150','150To200','200To250',
                                                                                '250To300','300To350','350To400','400To450','450To500',
                                                                                '500To550','550To600','600To1000'])
Train_ManipCompleted_1_Continuous['PRI.OVERDUE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.OVERDUE.ACCTS'],
                                                                PRI_OVERDUE_ACCTS_bucket_array,include_lowest=True,
                                                               labels = ['0To5','5To10','10To15','15To20','20To25','25To30'])
Train_ManipCompleted_1_Continuous['SEC.NO.OF.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.NO.OF.ACCTS'],
                                                              SEC_NO_OF_ACCTS_bucket_array,include_lowest=True,
                                                             labels = ['0To10','10To20','20To30','30To40','40To50','50To60','60To70','70To80',
                                                           '80To90','90To100'])
Train_ManipCompleted_1_Continuous['SEC.ACTIVE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.ACTIVE.ACCTS'],
                                                               SEC_ACTIVE_ACCTS_bucket_array,include_lowest=True,
                                                              labels = ['0To10','10To20','20To30','30To40','40To50','50To60','60To70','70To80',
                                                           '80To90','90To100'])
Train_ManipCompleted_1_Continuous['SEC.OVERDUE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.OVERDUE.ACCTS'],
                                                                SEC_OVERDUE_ACCTS_bucket_array,include_lowest=True,
                                                                labels = ['0To5','5To10','10To15','15To20','20To25','25To30'])
Train_ManipCompleted_1_Continuous['AvgAcctAgeInMonths'] = pd.cut(Train_ManipCompleted_1_Continuous['AvgAcctAgeInMonths'],
                                                                 AvgAcctAgeInMonths_bucket_array,include_lowest=True,
                                                                 labels = ['0To50','50To100','100To150','150To200','200To250',
                                                                                '250To300','300To350','350To400','400To450','450To500',
                                                                                '500To550','550To600','600To1000'])
Train_ManipCompleted_1_Continuous['CredHistLenInMonts'] = pd.cut(Train_ManipCompleted_1_Continuous['CredHistLenInMonts'],
                                                                 CredHistLenInMonts_bucket_array,include_lowest=True,
                                                                 labels = ['0To50','50To100','100To150','150To200','200To250',
                                                                                '250To300','300To350','350To400','400To450','450To500',
                                                                                '500To550','550To600','600To1000'])

In [151]:
Train_ManipCompleted_1_Continuous['disbursed_amount'] = pd.cut(Train_ManipCompleted_1_Continuous['disbursed_amount'],
                                                               disbursed_amount_bucket_array,include_lowest=True,
                                                              labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
Train_ManipCompleted_1_Continuous['asset_cost'] = pd.cut(Train_ManipCompleted_1_Continuous['asset_cost'],
                                                         asset_cost_bucket_array,include_lowest=True,
                                                        labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
Train_ManipCompleted_1_Continuous['ltv'] = pd.cut(Train_ManipCompleted_1_Continuous['ltv'],ltv_bucket_array,include_lowest=True,
                                                 labels = [1,2,3,4,5,6,7,8,9,10])
Train_ManipCompleted_1_Continuous['Age'] = pd.cut(Train_ManipCompleted_1_Continuous['Age'],Age_bucket_array,include_lowest=True,
                                                 labels = [1,2,3,4,5,6,7,8,9,10,11])
Train_ManipCompleted_1_Continuous['HowManyDaysSinceDisburse'] = pd.cut(Train_ManipCompleted_1_Continuous['HowManyDaysSinceDisburse'],
                                                                       HowManyDaysSinceDisburse,include_lowest=True,
                                                                      labels = [1,2,3,4,5,6,7,8,9,10,11,12,13])
Train_ManipCompleted_1_Continuous['PERFORM_CNS.SCORE'] = pd.cut(Train_ManipCompleted_1_Continuous['PERFORM_CNS.SCORE'],
                                                                PERFORM_CNS_SCORE_bucket_array,include_lowest=True,
                                                               labels = [1,2,3,4,5,6,7,8,9,10,11])
Train_ManipCompleted_1_Continuous['PRI.NO.OF.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.NO.OF.ACCTS'],
                                                              PRI_NO_OF_ACCTS_bucket_array,include_lowest=True,
                                                             labels = [1,2,3,4,5,6,7,8,9,10,11,12,13])
Train_ManipCompleted_1_Continuous['PRI.ACTIVE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.ACTIVE.ACCTS'],
                                                               PRI_ACTIVE_ACCTS_bucket_array,include_lowest=True,
                                                              labels = [1,2,3,4,5,6,7,8,9,10,11,12,13])
Train_ManipCompleted_1_Continuous['PRI.OVERDUE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['PRI.OVERDUE.ACCTS'],
                                                                PRI_OVERDUE_ACCTS_bucket_array,include_lowest=True,
                                                               labels = [1,2,3,4,5,6])
Train_ManipCompleted_1_Continuous['SEC.NO.OF.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.NO.OF.ACCTS'],
                                                              SEC_NO_OF_ACCTS_bucket_array,include_lowest=True,
                                                             labels = [1,2,3,4,5,6,7,8,9,10])
Train_ManipCompleted_1_Continuous['SEC.ACTIVE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.ACTIVE.ACCTS'],
                                                               SEC_ACTIVE_ACCTS_bucket_array,include_lowest=True,
                                                              labels = [1,2,3,4,5,6,7,8,9,10])
Train_ManipCompleted_1_Continuous['SEC.OVERDUE.ACCTS'] = pd.cut(Train_ManipCompleted_1_Continuous['SEC.OVERDUE.ACCTS'],
                                                                SEC_OVERDUE_ACCTS_bucket_array,include_lowest=True,
                                                                labels = [1,2,3,4,5,6])
Train_ManipCompleted_1_Continuous['AvgAcctAgeInMonths'] = pd.cut(Train_ManipCompleted_1_Continuous['AvgAcctAgeInMonths'],
                                                                 AvgAcctAgeInMonths_bucket_array,include_lowest=True,
                                                                 labels = [1,2,3,4,5,6,7,8,9,10,11,12,13])
Train_ManipCompleted_1_Continuous['CredHistLenInMonts'] = pd.cut(Train_ManipCompleted_1_Continuous['CredHistLenInMonts'],
                                                                 CredHistLenInMonts_bucket_array,include_lowest=True,
                                                                 labels = [1,2,3,4,5,6,7,8,9,10,11,12,13])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [152]:
Train_ManipCompleted_1_Continuous['Age'].isnull().sum()

19866

ZeroTo3lacs = {'0To10000' : '1',
               '10000To20000': '2',
               '20000To30000': '3',
               '30000To40000': '4',
               '40000To50000': '5',
               '50000To60000': '6',
               '60000To70000': '7',
               '70000To80000': '8',
               '80000To90000': '9',
               '90000To100000': '10',
               '100000To125000': '11',
               '125000To150000': '12',
               '150000To200000': '13',
               '200000To250000': '14',
               '250000To300000': '15'}
NinetyTo100 = {'0To10': '1',
              '10To20': '2',
              '20To30': '3',
              '30To40': '4',
              '40To50': '5',
              '50To60': '6',
              '60To70': '7',
              '70To80': '8',
              '80To90': '9',
              '90To100': '10'}
NinetyT0110 = {'0To10': '1',
              '10To20': '2',
              '20To30': '3',
              '30To40': '4',
              '40To50': '5',
              '50To60': '6',
              '60To70': '7',
              '70To80': '8',
              '80To90': '9',
              '90To100': '10',
              '100To110': '11'}
ZeroTo2000 = {'0To100': '1',
              '100To200': '2',
              '200To300': '3',
              '300To400': '4',
              '400To500': '5',
              '500To600': '6',
              '600To700': '7',
              '700To800': '8',
              '800To900': '9',
              '900To1000': '10',
              '1000To2000': '11'}
ZeroTo1000 = {'0To50': '1',
              '50To100': '2',
              '100To150': '3',
              '150To200': '4',
              '200To250': '5',
              '250To300': '6',
              '300To350': '7',
              '350To400': '8',
              '400To450': '9',
              '450To500': '10',
              '500To550': '11',
              '550To600': '12',
              '600To1000': '13'}
ZeroTo30 = {'0To5': '1',
            '5To10': '2',
            '10To15': '3',
            '15To20': '4',
            '20To25': '5',
            '25To30': '6'}

In [153]:
Train_ManipCompleted_1_Continuous.head()

,disbursed_amount,asset_cost,ltv,Age,HowManyDaysSinceDisburse,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AvgAcctAgeInMonths,CredHistLenInMonts,NO.OF_INQUIRIES
0,6,6,9,4,9,1,1,1,1,0,...,0,0,0,0,0,0,0,1,1,0
1,5,7,8,4,5,6,1,1,1,27600,...,0,0,0,1991,0,0,1,1,1,0
2,6,7,9,4,10,1,1,1,1,0,...,0,0,0,0,0,0,0,1,1,0
3,6,7,9,3,4,4,1,1,1,0,...,0,0,0,31,0,0,0,1,1,1
4,6,7,9,5,5,1,1,1,1,0,...,0,0,0,0,0,0,0,1,1,1


In [88]:
Train_ManipCompleted_1_Continuous.dtypes

disbursed_amount                       category
asset_cost                             category
ltv                                    category
Age                                    category
HowManyDaysSinceDisburse               category
PERFORM_CNS.SCORE                      category
PRI.NO.OF.ACCTS                        category
PRI.ACTIVE.ACCTS                       category
PRI.OVERDUE.ACCTS                      category
PRI.CURRENT.BALANCE                       int64
PRI.SANCTIONED.AMOUNT                     int64
PRI.DISBURSED.AMOUNT                      int64
SEC.NO.OF.ACCTS                        category
SEC.ACTIVE.ACCTS                       category
SEC.OVERDUE.ACCTS                      category
SEC.CURRENT.BALANCE                       int64
SEC.SANCTIONED.AMOUNT                     int64
SEC.DISBURSED.AMOUNT                      int64
PRIMARY.INSTAL.AMT                        int64
SEC.INSTAL.AMT                            int64
NEW.ACCTS.IN.LAST.SIX.MONTHS            

In [154]:
# Scaling the data with MinmaxScaler for required columns
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler = MinMaxScaler()

Train_ManipCompleted_1_Continuous_ScalingColumns = Train_ManipCompleted_1_Continuous[['PRI.CURRENT.BALANCE',
                                                                                      'PRI.SANCTIONED.AMOUNT',
                                                                                      'PRI.DISBURSED.AMOUNT',
                                                                                      'SEC.CURRENT.BALANCE',
                                                                                      'SEC.SANCTIONED.AMOUNT',
                                                                                      'SEC.DISBURSED.AMOUNT',
                                                                                      'PRIMARY.INSTAL.AMT',
                                                                                      'SEC.INSTAL.AMT']]

In [155]:
Train_ManipCompleted_1_Continuous_ScalingColumns =pd.DataFrame(MinMaxScaler.fit_transform(Train_ManipCompleted_1_Continuous_ScalingColumns), 
                                                                                         columns = ['PRI.CURRENT.BALANCE',
                                                                                      'PRI.SANCTIONED.AMOUNT',
                                                                                      'PRI.DISBURSED.AMOUNT',
                                                                                      'SEC.CURRENT.BALANCE',
                                                                                      'SEC.SANCTIONED.AMOUNT',
                                                                                      'SEC.DISBURSED.AMOUNT',
                                                                                      'PRIMARY.INSTAL.AMT',
                                                                                      'SEC.INSTAL.AMT'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [173]:
Train_ManipCompleted_1_Continuous_final = pd.concat([Train_ManipCompleted_1_Continuous.drop(columns = ['PRI.CURRENT.BALANCE',
                                                                                      'PRI.SANCTIONED.AMOUNT',
                                                                                      'PRI.DISBURSED.AMOUNT',
                                                                                      'SEC.CURRENT.BALANCE',
                                                                                      'SEC.SANCTIONED.AMOUNT',
                                                                                      'SEC.DISBURSED.AMOUNT',
                                                                                      'PRIMARY.INSTAL.AMT',
                                                                                      'SEC.INSTAL.AMT']),
                                                     Train_ManipCompleted_1_Continuous_ScalingColumns],axis=1)

In [174]:
len(Train_ManipCompleted_1_Continuous_final.columns)

25

In [175]:
Train_ManipCompleted_1_Continuous_final.head()

,disbursed_amount,asset_cost,ltv,Age,HowManyDaysSinceDisburse,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,SEC.NO.OF.ACCTS,...,CredHistLenInMonts,NO.OF_INQUIRIES,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT
0,6,6,9,4,9,1,1,1,1,1,...,1,0,0.064710,0.00000,0.00000,0.015698,0.0,0.0,0.000000,0.0
1,5,7,8,4,5,6,1,1,1,1,...,1,0,0.064978,0.00005,0.00005,0.015698,0.0,0.0,0.000078,0.0
2,6,7,9,4,10,1,1,1,1,1,...,1,0,0.064710,0.00000,0.00000,0.015698,0.0,0.0,0.000000,0.0
3,6,7,9,3,4,4,1,1,1,1,...,1,1,0.064710,0.00000,0.00000,0.015698,0.0,0.0,0.000001,0.0
4,6,7,9,5,5,1,1,1,1,1,...,1,1,0.064710,0.00000,0.00000,0.015698,0.0,0.0,0.000000,0.0


In [176]:
Train_ManipCompleted_2 = pd.concat([Train_ManipCompleted_1_Continuous_final,Train_ManipCompleted_1_Categorical],axis=1)

In [177]:
len(Train_ManipCompleted_2.columns)

41

In [178]:
Train_ManipCompleted_2.head()

,disbursed_amount,asset_cost,ltv,Age,HowManyDaysSinceDisburse,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,SEC.NO.OF.ACCTS,...,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,loan_default,EmploymentType_Salaried,EmploymentType_Self employed,Bureau_score_description
0,6,6,9,4,9,1,1,1,1,1,...,1,1,0,0,0,0,0,1,0,1
1,5,7,8,4,5,6,1,1,1,1,...,1,1,0,0,0,0,1,0,1,7
2,6,7,9,4,10,1,1,1,1,1,...,1,1,0,0,0,0,0,0,1,1
3,6,7,9,3,4,4,1,1,1,1,...,1,1,0,0,0,0,1,0,1,2
4,6,7,9,5,5,1,1,1,1,1,...,1,1,0,0,0,0,1,0,1,1


In [179]:
Train_ManipCompleted_2.isnull().sum()

disbursed_amount                           6
asset_cost                                 8
ltv                                        0
Age                                    19866
HowManyDaysSinceDisburse                   0
PERFORM_CNS.SCORE                          0
PRI.NO.OF.ACCTS                            0
PRI.ACTIVE.ACCTS                           0
PRI.OVERDUE.ACCTS                          0
SEC.NO.OF.ACCTS                            0
SEC.ACTIVE.ACCTS                           0
SEC.OVERDUE.ACCTS                          0
NEW.ACCTS.IN.LAST.SIX.MONTHS               0
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS        0
AvgAcctAgeInMonths                         0
CredHistLenInMonts                         0
NO.OF_INQUIRIES                            0
PRI.CURRENT.BALANCE                        0
PRI.SANCTIONED.AMOUNT                      0
PRI.DISBURSED.AMOUNT                       0
SEC.CURRENT.BALANCE                        0
SEC.SANCTIONED.AMOUNT                      0
SEC.DISBUR

In [180]:
Train_ManipCompleted_2.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'Age',
       'HowManyDaysSinceDisburse', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'branch_id', 'supplier_id', 'manufacturer_id',
       'Current_pincode_ID', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description'],
      dtype='object')

In [201]:
#Train_ManipCompleted_2.isnull().sum()

In [210]:
Train_ManipCompleted_2 = Train_ManipCompleted_2[Train_ManipCompleted_2['disbursed_amount'].isnull()==False]
Train_ManipCompleted_2 = Train_ManipCompleted_2[Train_ManipCompleted_2['disbursed_amount'].isnull()==False]
Train_ManipCompleted_2 = Train_ManipCompleted_2[Train_ManipCompleted_2['asset_cost'].isnull()==False]

In [211]:
Train_ManipCompleted_2.isnull().sum()

disbursed_amount                           0
asset_cost                                 0
ltv                                        0
Age                                    19866
HowManyDaysSinceDisburse                   0
PERFORM_CNS.SCORE                          0
PRI.NO.OF.ACCTS                            0
PRI.ACTIVE.ACCTS                           0
PRI.OVERDUE.ACCTS                          0
SEC.NO.OF.ACCTS                            0
SEC.ACTIVE.ACCTS                           0
SEC.OVERDUE.ACCTS                          0
NEW.ACCTS.IN.LAST.SIX.MONTHS               0
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS        0
AvgAcctAgeInMonths                         0
CredHistLenInMonts                         0
NO.OF_INQUIRIES                            0
PRI.CURRENT.BALANCE                        0
PRI.SANCTIONED.AMOUNT                      0
PRI.DISBURSED.AMOUNT                       0
SEC.CURRENT.BALANCE                        0
SEC.SANCTIONED.AMOUNT                      0
SEC.DISBUR

In [212]:
IndeVectors_Scaled_Bucketed = Train_ManipCompleted_2.drop(columns = ['loan_default','Age'])
Lables_Scaled_Bucketed = Train_ManipCompleted_2['loan_default']

In [213]:
#IndeVectors_Scaled_Bucketed.dtypes

In [214]:
len(IndeVectors_Scaled_Bucketed.columns)

39

In [215]:
IndeVectors_Scaled_Bucketed.isnull().sum()

disbursed_amount                       0
asset_cost                             0
ltv                                    0
HowManyDaysSinceDisburse               0
PERFORM_CNS.SCORE                      0
PRI.NO.OF.ACCTS                        0
PRI.ACTIVE.ACCTS                       0
PRI.OVERDUE.ACCTS                      0
SEC.NO.OF.ACCTS                        0
SEC.ACTIVE.ACCTS                       0
SEC.OVERDUE.ACCTS                      0
NEW.ACCTS.IN.LAST.SIX.MONTHS           0
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS    0
AvgAcctAgeInMonths                     0
CredHistLenInMonts                     0
NO.OF_INQUIRIES                        0
PRI.CURRENT.BALANCE                    0
PRI.SANCTIONED.AMOUNT                  0
PRI.DISBURSED.AMOUNT                   0
SEC.CURRENT.BALANCE                    0
SEC.SANCTIONED.AMOUNT                  0
SEC.DISBURSED.AMOUNT                   0
PRIMARY.INSTAL.AMT                     0
SEC.INSTAL.AMT                         0
branch_id       

In [216]:
trainBucket_x,validBucket_x,trainBucket_y,validBucket_y = model_selection.train_test_split(IndeVectors_Scaled_Bucketed,Lables_Scaled_Bucketed,test_size = 0.1,random_state = 9999)

In [217]:
# Logistic Regression with Scaled data

from sklearn.linear_model import LogisticRegression

LR1_Scaled = LogisticRegression(class_weight = 'balanced')
LR1_Scaled.fit(trainBucket_x,trainBucket_y)
ypred_LR1Scaled = LR1_Scaled.predict(validBucket_x)
print("AUC of Logistic Regression with Scaled data: ",metrics.roc_auc_score(validBucket_y,ypred_LR1Scaled))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC of Logistic Regression with Scaled data:  0.58159732201047


In [47]:
# Ridge Classification with Scaled data

from sklearn.linear_model import RidgeClassifier

RC1_Scaled = RidgeClassifier(alpha=0.002, fit_intercept=True, 
                                    normalize=True, class_weight='balanced', solver= 'auto', random_state=1234)
RC1_Scaled.fit(trainScaled_x,trainScaled_y)
ypred_RC1_Scaled = RC1_Scaled.predict(validScaled_x)
print("AUC of Ridge Classification with Scaled data: ",metrics.roc_auc_score(validScaled_y,ypred_RC1_Scaled))

AUC of Ridge Classification with Scaled data:  0.5904150716864373


In [34]:
# Random forest with Scaled data

RF1_Scaled = RandomForestClassifier(n_estimators=300,bootstrap=True,max_features='sqrt')
RF1_Scaled.fit(trainScaled_x,trainScaled_y)
ypred_RF1_Scaled = RF1_Scaled.predict(validScaled_x)
print("AUC of Random forest with Scaled data: ",metrics.roc_auc_score(validScaled_y,ypred_RF1_Scaled))

AUC of Ridge Classification with Scaled data:  0.5113495426675241


In [36]:
feature_importances = pd.DataFrame(RF1_Scaled.feature_importances_,index = trainScaled_x.columns,
                                   columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
ltv,0.095252
disbursed_amount,0.088741
asset_cost,0.085965
Current_pincode_ID,0.083590
HowManyDaysSinceDisburse,0.078173
Employee_code_ID,0.075330
Age,0.073152
supplier_id,0.068746
branch_id,0.039859
State_ID,0.025027


In [40]:
feature_importances[feature_importances['importance']>=0.01].index

Index(['ltv', 'disbursed_amount', 'asset_cost', 'Current_pincode_ID',
       'HowManyDaysSinceDisburse', 'Employee_code_ID', 'Age', 'supplier_id',
       'branch_id', 'State_ID', 'PERFORM_CNS.SCORE', 'CredHistLenInMonts',
       'PRI.CURRENT.BALANCE', 'AvgAcctAgeInMonths', 'manufacturer_id',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'PRI.NO.OF.ACCTS', 'Bureau_score_description', 'NO.OF_INQUIRIES'],
      dtype='object')

In [41]:
trainx_important = trainScaled_x[['ltv', 'disbursed_amount', 'asset_cost', 'Current_pincode_ID',
       'HowManyDaysSinceDisburse', 'Employee_code_ID', 'Age', 'supplier_id',
       'branch_id', 'State_ID', 'PERFORM_CNS.SCORE', 'CredHistLenInMonts',
       'PRI.CURRENT.BALANCE', 'AvgAcctAgeInMonths', 'manufacturer_id',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'PRI.NO.OF.ACCTS', 'Bureau_score_description', 'NO.OF_INQUIRIES']]

In [43]:
# Logistic Regression with important columns of Random Forest

LR2_Scaled = LogisticRegression(class_weight = 'balanced')
LR2_Scaled.fit(trainx_important,trainScaled_y)
ypred_LR2Scaled = LR2_Scaled.predict(validScaled_x[['ltv', 'disbursed_amount', 'asset_cost', 'Current_pincode_ID',
       'HowManyDaysSinceDisburse', 'Employee_code_ID', 'Age', 'supplier_id',
       'branch_id', 'State_ID', 'PERFORM_CNS.SCORE', 'CredHistLenInMonts',
       'PRI.CURRENT.BALANCE', 'AvgAcctAgeInMonths', 'manufacturer_id',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'PRI.NO.OF.ACCTS', 'Bureau_score_description', 'NO.OF_INQUIRIES']])
print("AUC of Logistic Regression with Scaled data: ",metrics.roc_auc_score(validScaled_y,ypred_LR2Scaled))

C:\Users\chait\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC of Logistic Regression with Scaled data:  0.5852472034372354


In [48]:
#Trying out Logistic Regression and Ridge Classification with Scaled and SMOTE data

from imblearn.over_sampling import SMOTE
smt = SMOTE()
trainScaled_x_Smote, trainScaled_y_Smote = smt.fit_sample(trainScaled_x, trainScaled_y)

In [50]:
# Building & Predicting with scaled and smoted data

LR3_Scaled_Smote = LogisticRegression(class_weight = 'balanced')
LR3_Scaled_Smote.fit(trainScaled_x_Smote,trainScaled_y_Smote)
ypred_LR3Scaled_Smote = LR3_Scaled_Smote.predict(validScaled_x)
print("AUC of Logistic Regression with Scaled and SMOTE data: ",metrics.roc_auc_score(validScaled_y,ypred_LR3Scaled_Smote))


RC2_Scaled_Smote = RidgeClassifier(alpha=0.0531, fit_intercept=True, 
                                    normalize=True, class_weight='balanced', solver= 'auto', random_state=1234)
RC2_Scaled_Smote.fit(trainScaled_x_Smote,trainScaled_y_Smote)
ypred_RC3_Scaled_Smote = RC2_Scaled_Smote.predict(validScaled_x)
print("AUC of Ridge Classification with Scaled & Smote data: ",metrics.roc_auc_score(validScaled_y,ypred_RC3_Scaled_Smote))

C:\Users\chait\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC of Logistic Regression with Scaled and SMOTE data:  0.5829787453683546
AUC of Ridge Classification with Scaled & Smote data:  0.5911995966401621


In [90]:
TotalPred_LR3_Scaled_Smote = LR1_Scaled.predict(IndeVectors_Scaled)
TotalPred_RC2_Scaled_Smote = RC1_Scaled.predict(IndeVectors_Scaled)

In [91]:
print(len(IndeVectors_Scaled))
print(len(TotalPred_LR3_Scaled_Smote))
print(len(TotalPred_RC2_Scaled_Smote))
print(len(Lables_Scaled))

233154
233154
233154
233154


In [53]:
print(type(TotalPred_LR3_Scaled_Smote))
print(type(TotalPred_RC2_Scaled_Smote))
print(type(Lables_Scaled))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [92]:
LR3AndRc3OutputsForStacking_NN = pd.concat([pd.DataFrame(TotalPred_LR3_Scaled_Smote),pd.DataFrame(TotalPred_RC2_Scaled_Smote),
                                           Lables_Scaled],axis=1,ignore_index= True)

In [93]:
LR3AndRc3OutputsForStacking_NN.columns = ['LR3_loan_default','RC2_loan_default','Actual_loan_default']

In [94]:
LR3AndRc3OutputsForStacking_NN.head()

,LR3_loan_default,RC2_loan_default,Actual_loan_default
0,1,1,0
1,1,1,1
2,1,1,0
3,1,1,1
4,1,1,1


In [95]:
LR3_RC2PredictedSame = []
for i in range(len(LR3AndRc3OutputsForStacking_NN)):
    if (LR3AndRc3OutputsForStacking_NN['LR3_loan_default'][i] - LR3AndRc3OutputsForStacking_NN['RC2_loan_default'][i] == 0):
        LR3_RC2PredictedSame.append(0)
    else:
        LR3_RC2PredictedSame.append(1)

In [96]:
print(len(LR3AndRc3OutputsForStacking_NN))
print(len(LR3_RC2PredictedSame))

233154
233154


In [97]:
LR3AndRc3OutputsForStacking_NN['LR3_RC2PredictedSame'] = LR3_RC2PredictedSame

In [98]:
LR3AndRc3OutputsForStacking_NN.head()

,LR3_loan_default,RC2_loan_default,Actual_loan_default,LR3_RC2PredictedSame
0,1,1,0,0
1,1,1,1,0
2,1,1,0,0
3,1,1,1,0
4,1,1,1,0


In [99]:
stacker_Inde = LR3AndRc3OutputsForStacking_NN.drop(labels = 'Actual_loan_default',axis=1)
stacker_Labels = LR3AndRc3OutputsForStacking_NN['Actual_loan_default']

In [101]:
stacker_Labels_cat = to_categorical(stacker_Labels)

In [102]:
Model_NN_Stacker = Sequential()

# Getting No.Of Columns 
ncols = stacker_Inde.shape[1]

# Defining layers
Model_NN_Stacker.add(Dense(250, activation = 'relu',input_shape = (ncols,)))
Model_NN_Stacker.add(Dense(250, activation = 'relu'))
Model_NN_Stacker.add(Dense(250, activation = 'relu'))
Model_NN_Stacker.add(Dense(2,activation='softmax'))
Model_NN_Stacker.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = [auc])
Model_NN_Stacker.fit(stacker_Inde,stacker_Labels_cat,epochs=30,validation_split=0.10,callbacks=[early_stopping_monitor])

Train on 209838 samples, validate on 23316 samples
Epoch 1/30
209838/209838 [==============================] - 21s 98us/step - loss: 0.5106 - auc: 0.8159 - val_loss: 0.5298 - val_auc: 0.8160
Epoch 2/30
209838/209838 [==============================] - 22s 104us/step - loss: 0.5095 - auc: 0.8153 - val_loss: 0.5294 - val_auc: 0.8156
Epoch 3/30
209838/209838 [==============================] - 20s 97us/step - loss: 0.5093 - auc: 0.8158 - val_loss: 0.5287 - val_auc: 0.8157
Epoch 4/30
209838/209838 [==============================] - 23s 110us/step - loss: 0.5093 - auc: 0.8157 - val_loss: 0.5287 - val_auc: 0.8157
Epoch 5/30
209838/209838 [==============================] - 20s 97us/step - loss: 0.5093 - auc: 0.8156 - val_loss: 0.5288 - val_auc: 0.8156
Epoch 6/30
209838/209838 [==============================] - 20s 97us/step - loss: 0.5092 - auc: 0.8155 - val_loss: 0.5291 - val_auc: 0.8156
Epoch 7/30
209838/209838 [==============================] - 21s 100us/step - loss: 0.5092 - auc: 0.8155 - v

In [81]:
LR3Predicted_Correctly = []
for i in range(len(LR3AndRc3OutputsForStacking_NN)):
    if (LR3AndRc3OutputsForStacking_NN['LR3_loan_default'][i] - LR3AndRc3OutputsForStacking_NN['Actual_loan_default'][i] == 0):
        LR3Predicted_Correctly.append(0)
    else:
        LR3Predicted_Correctly.append(1)

In [83]:
len(LR3Predicted_Correctly)

233154

In [84]:
RC2Predicted_Correctly = []
for i in range(len(LR3AndRc3OutputsForStacking_NN)):
    if (LR3AndRc3OutputsForStacking_NN['RC2_loan_default'][i] - LR3AndRc3OutputsForStacking_NN['Actual_loan_default'][i] == 0):
        RC2Predicted_Correctly.append(0)
    else:
        RC2Predicted_Correctly.append(1)

In [86]:
len(RC2Predicted_Correctly)

233154

In [85]:
BothPredictorsPredictedCorrectly = []
for i in range(len(LR3AndRc3OutputsForStacking_NN)):
    if(LR3AndRc3OutputsForStacking_NN['LR3_RC2PredictedSame'][i] == 0):
        if(LR3AndRc3OutputsForStacking_NN['LR3_loan_default'][i] - LR3AndRc3OutputsForStacking_NN['Actual_loan_default'][i] == 0):
            BothPredictorsPredictedCorrectly.append("BothPrecitedCorrect")
        else:
            BothPredictorsPredictedCorrectly.append("BothPrecitedWrog")
        
    else:
        if(LR3AndRc3OutputsForStacking_NN['LR3_loan_default'][i] - LR3AndRc3OutputsForStacking_NN['Actual_loan_default'][i] == 0):
            BothPredictorsPredictedCorrectly.append("LR3PredictedCorrect")
        else:
            BothPredictorsPredictedCorrectly.append("RC2PredictedCorrect")        

In [87]:
len(BothPredictorsPredictedCorrectly)

233154

In [76]:
LR3_RC2PredictedSame

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [62]:
from keras.callbacks import EarlyStopping

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [63]:
IV4NN = LR3AndRc3OutputsForStacking_NN.drop(labels = 'Actual_loan_default',axis=1)
L4NN  = LR3AndRc3OutputsForStacking_NN['Actual_loan_default']

from keras.utils import to_categorical
L4NN_cat = to_categorical(L4NN)
print(L4NN_cat[0:10])

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [66]:
# Building a Neural Network 

Model_Stacked_3_2 = Sequential()
ncols = IV4NN.shape[1]

Model_Stacked_3_2.add(Dense(108,activation = 'relu',input_shape=(ncols,)))
Model_Stacked_3_2.add(Dense(2,activation = 'softmax'))

Model_Stacked_3_2.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = [auc])
Model_Stacked_3_2.fit(IV4NN,L4NN_cat,epochs=30,validation_split=0.10,callbacks=[early_stopping_monitor])

Train on 209838 samples, validate on 23316 samples
Epoch 1/30
209838/209838 [==============================] - 21s 100us/step - loss: 0.5112 - auc: 0.8118 - val_loss: 0.5305 - val_auc: 0.8160
Epoch 2/30
209838/209838 [==============================] - 20s 96us/step - loss: 0.5094 - auc: 0.8156 - val_loss: 0.5283 - val_auc: 0.8161
Epoch 3/30
209838/209838 [==============================] - 20s 95us/step - loss: 0.5092 - auc: 0.8158 - val_loss: 0.5303 - val_auc: 0.8162
Epoch 4/30
209838/209838 [==============================] - 21s 99us/step - loss: 0.5093 - auc: 0.8160 - val_loss: 0.5309 - val_auc: 0.8161
Epoch 5/30
209838/209838 [==============================] - 20s 96us/step - loss: 0.5091 - auc: 0.8161 - val_loss: 0.5284 - val_auc: 0.8161


In [67]:
# Modifying data manipulations function to work for test data as it doesn't have labels column
# Function for doing all the data manipulations
def AllDataManipulations_forTest(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = to_ordinal(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],ColumnOrdinalDictionary=Bureau_score_description)
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES']],temp1,pd.DataFrame(temp2,columns = ['Bureau_score_description'])],axis=1)
    return Manip_Data,Manip_Completed

In [68]:
Master_test = pd.read_csv("test_bqCt9Pv.csv")
Test_Data = Master_test.copy()
Test_ManipData,Test_ManipCompleted = AllDataManipulations_forTest(Test_Data)